# PyTorch 实现CO2排量预测
## 导入所需的库

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import yeojohnson, yeojohnson_normmax, yeojohnson_llf
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer

## 数据预处理
### 构造滑动窗口数据集

In [15]:
def create_sequencesCO2(window): 
    data = pd.read_csv('D:\Pycharm\chuan\data3.csv', encoding='gbk')
    # data = data['CO2排放总量']
    pt = PowerTransformer(method='yeo-johnson')
    data = pt.fit_transform(data['CO2排放总量'].values.reshape(-1, 1)) # 进行Yeo-Johnson变换
    sequences = [] 
    targets = [] 
    for i in range(len(data) - window):
        # window为3：seq：[1,2,3]
        seq = data[i:i + window] 
        target = data[i + window]
        # sequences:[[1,2,3],[]]
        sequences.append(seq) 
        targets.append(target)
    # 将numpy数组转换为PyTorch张量.x：（[[1,2,3],[]]）
    x=np.array(sequences)
    y=np.array(targets)
    print(x.shape,y.shape)
    print(x,y)
    return x, y, pt

## 定义数据集

In [16]:
class EmissionsDataset(Dataset):
    def __init__(self, x, y):
        self.features = x
        self.targets = y

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # 从数据集中获取单个样本的特征和目标，并将它们转换为 PyTorch 张量（Tensor）的格式，
        # 并添加一个维度作为批处理维度。
        # 特征：前几日排放量
        # 标签：后一天的CO2排放总量
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)
        return feature, target

## 神经网络
### RNN GRU LSTM FC

In [17]:
class RNNModel(nn.Module):
    # 定义RNN模型
    # hidden_dim：隐藏层维度
    # num_layers：RNN层数
    # input_dim：输入维度
    # output_dim：输出维度
    def __init__(self, hidden_dim=64, num_layers=2, input_dim=1, output_dim=1):
        super(RNNModel, self).__init__()
        # batch_first=True 表示输入数据的形状为 [batch_size, sequence_length, input_dim]
        # batch_size: 批处理大小，即一次处理的数据数量
        # sequence_length: 序列长度，即每个输入样本的长度,特征数量（NH3与CO2的排放量作为输入特征预测明天CO2排放量）= 2
        # input_dim: 输入维度，(今天的NH3排放量=1（size=1），昨天和今天的CO2排放量（size=2）)
        # shape:
        # [
        ##     [[8],[9,10]],
        #     [[2],[8,2]],
        #     [[3],[3,4]],
        # ]
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        # 全连接层
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
        )

    def forward(self, x):
        h0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out
    
class GRUModel(nn.Module):
    def __init__(self, hidden_dim=64, num_layers=2, input_dim=1, output_dim=1):
        super(GRUModel, self).__init__()
        # GRU层
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        # 创建一个多头注意力层，输入的特征维度为 hidden_dim，使用2个注意力头
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=2)
        # 层归一化层
        self.ln = nn.LayerNorm(hidden_dim)
        # 前馈神经网络，包括两层线性变换和一个ReLU激活函数
        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, hidden_dim)
        )
        # 线性层，用于将隐藏状态映射到输出维度
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # 初始化 GRU 隐藏状态h0,大小为 (num_layers, batch_size, hidden_dim)，并将其移动到输入 x 的设备上（CPU或GPU）
        h0 = torch.zeros(self.gru.num_layers, x.size(0), self.gru.hidden_size).to(x.device)
        # 通过GRU层：将输入 x 和初始隐藏状态 h0 传入GRU层，输出 out 是GRU的输出，第二个返回值是最后一个隐藏状态（在这里未使用）
        out, _ = self.gru(x, h0)
        # 将输出 out 传入注意力层，得到注意力加权后的输出 at_out 和注意力权重
        # at_out, _ = self.attention(out, out, out)
        # 将注意力加权后的输出与原始输出相加（残差连接），并进行层归一化
        # out = self.ln(out + at_out)
        # 将归一化后的输出传入前馈神经网络
        # out = self.ffn(out)
        # 将前馈神经网络的输出传入线性层，得到最终的输出
        # out = self.ln(out + at_out)
        # 提取最后一个时间步的输出 out[:, -1, :]，并通过全连接层得到最终输出
        out = self.fc(out[:, -1, :])
        return out
    
class LSTMModel(nn.Module):
    def __init__(self, hidden_dim=64, num_layers=2, input_dim=1, output_dim=1, bidirectional=False):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        # 防止过拟合
        # self.fc = nn.Sequential(nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, 32), nn.ReLU(),nn.Linear(32, output_dim))
        # self.batchnorm = nn.BatchNorm1d(hidden_dim * 2 if bidirectional else hidden_dim)
        # self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # 初始化 LSTM 隐藏状态
        h0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_dim).to(x.device)
        
        # LSTM 前向传播
        out, _ = self.lstm(x, (h0, c0))
        
        # 取最后一个时间步的输出，并送入全连接层
        out = self.fc(out[:, -1, :])
        # 防止过拟合
        # out = self.dropout(self.batchnorm(out[:, -1, :]))
        # out = self.fc(out)
        # return out
        return out

class FullyConnected(nn.Module):
    def __init__(self, hidden_dim=64, input_dim=1, output_dim=1, is_linear=False, window=1):
        super(FullyConnected, self).__init__()
        self.is_linear = is_linear
        self.window = window
        # 全连接层（线性层、激活层、线性层）
        self.fc1 = nn.Linear(input_dim * window, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # 单线性层
        self.only = nn.Linear(input_dim * window, output_dim)
    def forward(self, x):
        if self.is_linear:
            x = x.view(x.size(0), -1)
            x = self.only(x)
        else:
            x = x.view(x.size(0), -1)
            x = self.fc1(x)
            x = self.relu(x)
            x = self.fc2(x)
        return x

## Early Stopping

In [18]:
import os
import uuid
import glob

class EarlyStopping:
    def __init__(self, 
                 patience: int = 100,
                 min_delta: float = 0.00, 
                 path: str = None, 
                 monitor: str = 'val_loss', 
                 mode: str = 'min', 
                 save_top_k: int = 1,
                 dataset_name: str = None,
                 save_model: bool = False
                 ):
        """
        Args:
            patience (int): How many epochs to wait after last time validation loss improved.
            min_delta (float): Minimum change in the monitored quantity to qualify as an improvement.设置监控某个参数更新的最小值
            path (str): Path to save the best model.
            monitor (str): Quantity to be monitored. Default is 'val_loss'.
            mode (str): One of {'min', 'max'}. In 'min' mode, training will stop when the quantity monitored has stopped decreasing.（设为min：val_loss不减小时，停止训练）      
            save_top_k (int): Number of best models to save. Default is 1.
            dataset_name (str): Name of the dataset. Default is None.
            save_model (bool): Whether to save the model. Default is False.
        """
        self.patience = patience
        # 更新参数时的最小要求值
        self.min_delta = min_delta
        # 存储某个指标的最佳值（根据mode初始化为正负无穷）
        self.best_metrics = [float('inf')] if mode == 'min' else [float('-inf')]
        # 计算早停轮次
        self.counter = 0
        # 早停标志
        self.early_stop = False
        self.dataset_name = dataset_name
        # 监测指标
        self.monitor = monitor
        self.mode = mode
        # 保存的模型数量
        self.save_top_k = save_top_k
        # 定义更新的规则
        self.is_better = None
        self.sym = False
        self.save_model = save_model
        self.run_id = str(uuid.uuid4())  # Unique identifier for the training run

        # 最优模型地址
        if path:
            self.path = path
            self.sym = False
        else:
            self.path = None

        # 定义更新的规则
        if self.mode == 'min':
            self.is_better = lambda a, best: a < best
        elif self.mode == 'max':
            self.is_better = lambda a, best: a > best
        else:
            raise ValueError(f'Unknown mode: {self.mode}')
    
    # model：模型参数
    def __call__(self, val_metric, model):
        # 如果验证集损失小于当前最优
        if self.is_better(val_metric, self.best_metrics[-1] - self.min_delta):
            # 保存最佳
            self.best_metrics.append(val_metric)
            if self.save_model:
                self.save_checkpoint(model)
            print(f"{self.monitor} improved to {val_metric:.4f}")
            # 删除记录的非最优模型，重置计数器
            self._cleanup_checkpoints()
            self.counter = 0 
        else:
            # 如果验证集损失没有改善，则增加计数器
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        
        # 选出当前最优
        best_metric = min(self.best_metrics) if self.mode == 'min' else max(self.best_metrics)
        print(f"Current best {self.monitor}: {best_metric:.4f}")
        #     达瓦
    def add_path(self):
        if self.path is None:
            # monitor：验证损失。best_metrics[-1]：最后一个（最优）验证损失值。run_id：标识
            self.path = f'./pytorch_checkpoints/co2/{self.monitor}-{self.best_metrics[-1]:.4f}-{self.run_id}.pth'
            self.sym = True
        if self.sym:
            self.path = f'./pytorch_checkpoints/co2/{self.monitor}-{self.best_metrics[-1]:.4f}-{self.run_id}.pth'
        os.makedirs(os.path.dirname(self.path), exist_ok=True)

    def save_checkpoint(self, model):
        """Saves model when monitored metric improves."""
        self.add_path()
        torch.save(model.state_dict(), self.path)

    # 清除非最优模型
    def _cleanup_checkpoints(self):
        """Deletes all but the top k models for the current run instance."""
        # 查找当前目录下（./pytorch_checkpoints/co2/）所有符合特定模式的文件名。
        checkpoints = glob.glob(f'./pytorch_checkpoints/co2/{self.monitor}-*-{self.run_id}.pth')
        # 按照修改时间排序
        checkpoints.sort(key=os.path.getmtime)
        # 保留最新的k个文件，其余删除
        if len(checkpoints) > self.save_top_k:
            for ckpt in checkpoints[:-self.save_top_k]:
                os.remove(ckpt)

    def load_checkpoint(self, model):
        """Loads the saved model."""
        model.load_state_dict(torch.load(self.path))

## 定义超参数

In [19]:
import argparse
# python CO2_emissions.py --init_seed 42 --batch_size 64 --hidden_dim 128 --lr 1e-1 --num_layers 2 --num_heads 2 --max_epochs 50 --data_path "D:\Pycharm\chuan\data.csv"

parser = argparse.ArgumentParser(description='Carbon dioxide emissions with PyTorch Lightning')
parser.add_argument('--init_seed', type=int, default=42, help='Seed for initializing random number generators')
parser.add_argument('--batch_size', type=int, default=16, help='Batch size for training and evaluation')
parser.add_argument('--input_dim', type=int, default=1, help='Dimensionality of input features')
parser.add_argument('--hidden_dim', type=int, default=128, help='Dimensionality of hidden layers in RNN')
parser.add_argument('--lr', type=float, default=1e-3, help='Learning Rate for training the model')
parser.add_argument('--num_layers', type=int, default=2, help='Number of RNN layers')
parser.add_argument('--num_heads', type=int, default=2, help='Number of attention heads in Multi-Head Attention')
parser.add_argument('--max_epochs', type=int, default=100, help='Maximum number of epochs to train the model')
parser.add_argument('--data_path', type=str, default='D:\Pycharm\chuan\data3.csv', help='Path to the CSV data file')
# parser.add_argument('--data_path', type=str, default='D:\Pycharm\chuan\dataset\Video1.csv', help='Path to the CSV data file')
parser.add_argument('--patience', type=int, default=80, help='Patience for early stopping')
parser.add_argument('--num_workers', type=int, default=0, help='Number of workers for data loading')
parser.add_argument('--model_type', type=str, default='RNN', choices=['RNN', 'GRU', 'LSTM', 'FC'], help='Type of model to use')
parser.add_argument('--bidirectional', type=bool, default=True, help='Whether to use a bidirectional LSTM')
parser.add_argument('--window', type=int, default=4, help='The number of features')
parser.add_argument('--is_linear', type=bool, default=True, help='是否单线性层')
hparams, unknown = parser.parse_known_args()

## main

In [20]:
from rich.console import Console
from rich.text import Text
from rich.table import Table

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

init_seed = hparams.init_seed
torch.manual_seed(init_seed)
torch.cuda.manual_seed(init_seed)
torch.cuda.manual_seed_all(init_seed)
np.random.seed(init_seed)  # 用于numpy的随机数
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

x, y, pt = create_sequencesCO2(hparams.window)

if hparams.model_type == 'RNN':
    model = RNNModel(input_dim=hparams.input_dim, hidden_dim=hparams.hidden_dim, num_layers=hparams.num_layers)
elif hparams.model_type == 'GRU':
    model = GRUModel(input_dim=hparams.input_dim, hidden_dim=hparams.hidden_dim, num_layers=hparams.num_layers)
elif hparams.model_type == 'LSTM':
    model = LSTMModel(input_dim=hparams.input_dim, hidden_dim=hparams.hidden_dim, num_layers=hparams.num_layers, bidirectional = hparams.bidirectional)
elif hparams.model_type == 'FC':
    model = FullyConnected(input_dim=hparams.input_dim, hidden_dim=hparams.hidden_dim, is_linear=hparams.is_linear, window = hparams.window)
else:
    raise ValueError(f"Unknown model type: {hparams.model_type}")

model.to(device)
# 实例化数据集
dataset = EmissionsDataset(x, y)
# 划分数据集
train_val_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=hparams.init_seed)
train_dataset, val_dataset = train_test_split(train_val_dataset, test_size=0.2, random_state=hparams.init_seed)

# 实例化数据加载器
train_loader = DataLoader(train_dataset, batch_size=hparams.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=hparams.batch_size, shuffle=False)

# 实例化优化器和损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=hparams.lr)
# 损失函数：
# MAE：nn.L1Loss()
# MSE：nn.MSELoss()
# 交叉熵：nn.CrossEntropyLoss()
criterion = nn.L1Loss()
early_stopping = EarlyStopping(patience=hparams.patience, save_model=True)

# 简单早停：
# patience = 5
# counter = 0
# best_val_loss = float('inf')

for epoch in range(hparams.max_epochs):
    train_losses = []
    # 进入训练模式
    model.train()
    # 循环训练集数据加载器中的每个batch，将inputs、targets加载到模型中训练
    for batch in train_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        # 算出预测值
        pred = model(inputs)
        # 计算损失
        loss = criterion(pred, targets)
        # 清零梯度
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        # 记录每一个batch的损失
        train_losses.append(loss)

    # 计算一个epoch的平均损失
    avg_train_loss = sum(train_losses) / len(train_losses)

    # 在验证集上计算损失
    val_losses = []
    # 进入评估模式
    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            pred = model(inputs)
            loss = criterion(pred, targets)
            val_losses.append(loss)

    avg_val_loss = sum(val_losses) / len(val_losses)

    # 检查是否需要提前停止
    early_stopping(avg_val_loss, model)
    
    # 简单的早停：
    # if avg_val_loss < best_val_loss:
    #     best_val_loss = avg_val_loss
    #     counter = 0
    # else:     # 如果验证损失没有改善，增加计数器
    #     counter += 1
        
    # 打印训练和验证损失
    print('Epoch: [{epoch}][{num_epochs}]\t'
        'Loss {avg_train_loss:.4f}\t'
        'val_Loss {avg_val_loss:.4f}\n'.format(
        epoch=epoch + 1, num_epochs=hparams.max_epochs,
        avg_train_loss=avg_train_loss,
        avg_val_loss=avg_val_loss)
    )
    
    # 简单早停
    # if counter >= patience:
    #     print("Early stopping")
    #     break
        
    if early_stopping.early_stop:
        if epoch == hparams.max_epochs:
            print("max epoch reached")
        else:
            print("Early stopping")
        break

if early_stopping.save_model:
    early_stopping.load_checkpoint(model)

# 测试模型
test_losses = []
test_loader = DataLoader(test_dataset, batch_size=hparams.batch_size, shuffle=False)
model.eval()
with torch.no_grad():
    for batch in test_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        # 算出预测值
        pred = model(inputs)
        orig_pred = pt.inverse_transform(pred.cpu().numpy().reshape(-1, 1))
        orig_targets = pt.inverse_transform(targets.cpu().numpy().reshape(-1, 1))
        test_mae = mean_absolute_error(orig_pred, orig_targets)
        test_r2 = r2_score(orig_targets, orig_pred)
        test_losses.append(test_mae)

avg_test_loss = sum(test_losses) / len(test_losses)

print('-' * 100)
console = Console()
table = Table(title="Test Results")
table.add_column("Metrics", style="cyan")
table.add_column("Results", style="magenta")
table.add_row('test_mae', f"{avg_test_loss:.4f}")
table.add_row('test_r2', f"{test_r2:.4f}")

console.print(table)
print('test_mae', f"{avg_test_loss:.4f}")
print('test_r2', f"{test_r2:.4f}")
test_losses.clear()


(28, 4, 1) (28, 1)
[[[-1.88139381]
  [-0.15846261]
  [ 0.11676888]
  [-0.23812704]]

 [[-0.15846261]
  [ 0.11676888]
  [-0.23812704]
  [ 0.42940496]]

 [[ 0.11676888]
  [-0.23812704]
  [ 0.42940496]
  [ 0.15484801]]

 [[-0.23812704]
  [ 0.42940496]
  [ 0.15484801]
  [ 0.11994289]]

 [[ 0.42940496]
  [ 0.15484801]
  [ 0.11994289]
  [-0.53250898]]

 [[ 0.15484801]
  [ 0.11994289]
  [-0.53250898]
  [-0.46828704]]

 [[ 0.11994289]
  [-0.53250898]
  [-0.46828704]
  [-1.45531381]]

 [[-0.53250898]
  [-0.46828704]
  [-1.45531381]
  [-1.18394154]]

 [[-0.46828704]
  [-1.45531381]
  [-1.18394154]
  [-1.62624807]]

 [[-1.45531381]
  [-1.18394154]
  [-1.62624807]
  [-1.2836217 ]]

 [[-1.18394154]
  [-1.62624807]
  [-1.2836217 ]
  [-1.18481239]]

 [[-1.62624807]
  [-1.2836217 ]
  [-1.18481239]
  [-1.4993197 ]]

 [[-1.2836217 ]
  [-1.18481239]
  [-1.4993197 ]
  [-1.06998365]]

 [[-1.18481239]
  [-1.4993197 ]
  [-1.06998365]
  [-0.0548754 ]]

 [[-1.4993197 ]
  [-1.06998365]
  [-0.0548754 ]
  [ 0.496

        Test Results        
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metrics  ┃ Results       ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ test_mae │ 94083424.0000 │
│ test_r2  │ 0.8106        │
└──────────┴───────────────┘

test_mae 94083424.0000
test_r2 0.8106
